In this notebook, I validate my implementation of transformer components and GPT-2 small by:
- Comparing the behavior of a GPT-2 transformer block with my own implementation of a GPT-2 transformer block
- Comparing the behavior of the GPT-2 embedding components with my own implementation of the GPT-2 embedding components
- Comparing the behavior of the full GPT-2 model with my own implementation of the full GPT-2 model
- Sampling from my implementation of the full GPT-2 model

There are slight deviations between the outputs of my model and that of the original GPT-2 model. As shown below, this appears to be primarily due to the original GPT-2 model using a slightly different variant of GeLU from both me and PyTorch.

In [1]:
import torch as t
import sys
import os
notebook_path = os.path.abspath('')
project_root = os.path.join(notebook_path, '..')
sys.path.append(project_root)

## Comparing GPT2 transformer block (and components) with my implementation of a GPT2 transformer block

In [2]:
from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

In [3]:
import torch as t
from src.blocks import TransformerDecoderBlock
from src.models import GPT2SmallModel


gpt2_block = model.h[0]
my_block = GPT2SmallModel().transformer_blocks[0]
#my_block = TransformerDecoderBlock(768, 12, 3072, 'gelu', 'layer_norm', 'mlpblock')

my_block.norm_layer1.weight, my_block.norm_layer1.bias = gpt2_block.ln_1.weight, gpt2_block.ln_1.bias
my_block.norm_layer2.weight, my_block.norm_layer2.bias = gpt2_block.ln_2.weight, gpt2_block.ln_2.bias

my_block.mlp_block.linear1.weight, my_block.mlp_block.linear1.bias = t.nn.Parameter(gpt2_block.mlp.c_fc.weight.T), gpt2_block.mlp.c_fc.bias
my_block.mlp_block.linear2.weight, my_block.mlp_block.linear2.bias = t.nn.Parameter(gpt2_block.mlp.c_proj.weight.T), gpt2_block.mlp.c_proj.bias

(wq,wk,wv) = t.chunk(gpt2_block.attn.c_attn.weight.T, 3, dim=0)
(bq,bk,bv) = t.chunk(gpt2_block.attn.c_attn.bias, 3, dim=0)

my_block.mha_block.linear_q.weight, my_block.mha_block.linear_q.bias = t.nn.Parameter(wq), t.nn.Parameter(bq)
my_block.mha_block.linear_k.weight, my_block.mha_block.linear_k.bias = t.nn.Parameter(wk), t.nn.Parameter(bk)
my_block.mha_block.linear_v.weight, my_block.mha_block.linear_v.bias = t.nn.Parameter(wv), t.nn.Parameter(bv)
my_block.mha_block.linear_o.weight, my_block.mha_block.linear_o.bias = t.nn.Parameter(gpt2_block.attn.c_proj.weight.T), gpt2_block.attn.c_proj.bias

my_block.eval()
gpt2_block.eval()

print(gpt2_block)
print(my_block)

GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D()
    (c_proj): Conv1D()
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D()
    (c_proj): Conv1D()
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
TransformerDecoderBlock(
  (norm_layer1): LayerNorm()
  (norm_layer2): LayerNorm()
  (mha_block): MultiheadAttentionBlock(
    (linear_q): Linear()
    (linear_k): Linear()
    (linear_v): Linear()
    (linear_o): Linear()
    (dropout_layer_1): Dropout()
    (dropout_layer_2): Dropout()
  )
  (mlp_block): MLPBlock(
    (linear1): Linear()
    (linear2): Linear()
    (dropout_layer): Dropout()
  )
)


#### Comparing transformer blocks

In [6]:
x = t.randn((1,5,768))

seq_len = x.shape[-2]
att_mask = t.where(t.arange(seq_len).unsqueeze(1) < t.arange(seq_len), -t.inf, 0)

gpt2_block.eval()
my_block.eval()

print("GPT2 Transformer block")
print(gpt2_block(x)[0])
print("My transformer block")
print(my_block(x)[0])

t.allclose(gpt2_block(x)[0], my_block(x)[0], atol=1e-2)

GPT2 Transformer block
tensor([[[ 12.5908,  -6.6954,  -2.4058,  ...,   3.2182,  -2.4987,  -3.6320],
         [  9.7275,   0.5249,  -6.6079,  ...,   1.3819,   8.2928,   5.6790],
         [  0.4569,   0.6982,   3.8048,  ...,   2.3666,  -3.6816,  -4.9414],
         [  3.3743,  -8.5024,  -9.2940,  ...,  10.8106,   2.6719,   4.6105],
         [  2.0780,  -2.1215, -11.5871,  ...,  -9.3378,   8.6726,   2.5591]]],
       grad_fn=<AddBackward0>)
My transformer block
tensor([[[ 12.5898,  -6.6965,  -2.4064,  ...,   3.2177,  -2.5010,  -3.6342],
         [  9.7267,   0.5249,  -6.6076,  ...,   1.3793,   8.2916,   5.6779],
         [  0.4550,   0.6957,   3.8032,  ...,   2.3663,  -3.6817,  -4.9401],
         [  3.3734,  -8.5016,  -9.2952,  ...,  10.8108,   2.6710,   4.6087],
         [  2.0779,  -2.1221, -11.5879,  ...,  -9.3391,   8.6724,   2.5579]]],
       grad_fn=<AddBackward0>)


True

#### Comparing normalization

In [7]:
print("Normalization")

print("GPT2 Layer Norm 1")
print(gpt2_block.ln_1(x))
print("My Layer Norm 1")
print(my_block.norm_layer1(x))

print("GPT2 Layer Norm 2")
print(gpt2_block.ln_2(x))
print("My Layer Norm 2")
print(my_block.norm_layer2(x))

print(t.allclose(my_block.norm_layer1(x), gpt2_block.ln_1(x), atol=1e-7))
print(t.allclose(my_block.norm_layer2(x), gpt2_block.ln_2(x), atol=1e-7))


Normalization
GPT2 Layer Norm 1
tensor([[[-0.2053,  0.0078, -0.2287,  ..., -0.0264, -0.4718, -0.1814],
         [ 0.0331,  0.2437, -0.2329,  ..., -0.2384,  0.0220,  0.1560],
         [-0.1627, -0.0466, -0.0133,  ...,  0.2635,  0.1210,  0.2487],
         [-0.0273, -0.1822, -0.1676,  ...,  0.0253, -0.2074,  0.2309],
         [ 0.1170, -0.0384, -0.0365,  ..., -0.2040,  0.2045,  0.0376]]],
       grad_fn=<NativeLayerNormBackward0>)
My Layer Norm 1
tensor([[[-0.2053,  0.0078, -0.2287,  ..., -0.0264, -0.4718, -0.1814],
         [ 0.0331,  0.2437, -0.2329,  ..., -0.2384,  0.0220,  0.1560],
         [-0.1627, -0.0466, -0.0133,  ...,  0.2635,  0.1210,  0.2487],
         [-0.0273, -0.1822, -0.1676,  ...,  0.0253, -0.2074,  0.2309],
         [ 0.1170, -0.0384, -0.0365,  ..., -0.2040,  0.2045,  0.0376]]],
       grad_fn=<AddBackward0>)
GPT2 Layer Norm 2
tensor([[[-0.0758,  0.0103, -0.2172,  ..., -0.1221, -3.2162, -0.9391],
         [ 0.0640,  0.2817, -0.2228,  ..., -1.5645,  0.2612,  0.9701],
    

#### Comparing attention

In [10]:
print("Multi-head attention")
print("GPT2 Attention")
print(gpt2_block.attn(x)[0])
print("My attention")
print(my_block.mha_block(x, attention_mask=att_mask)[0])


print(t.allclose(my_block.mha_block(x, attention_mask=att_mask)[0], gpt2_block.attn(x)[0], atol=1e-4))

Multi-head attention
GPT2 Attention
tensor([[[ 1.1816e+01, -1.1000e+00,  4.8975e+00,  ..., -1.9040e-01,
           1.4437e-01, -7.6243e-01],
         [ 2.1397e+01,  3.3469e+00, -1.4271e+00,  ...,  1.2569e+00,
          -1.4903e+00, -2.0294e-03],
         [-8.0171e+00, -3.6174e+00,  3.2031e+00,  ...,  8.7513e-01,
          -9.2649e-01,  5.2257e-01],
         [-7.1662e+00, -2.0880e+01,  1.4626e+01,  ...,  9.2103e-01,
           9.0435e-01,  8.1361e-01],
         [ 5.4927e+00, -1.4809e+01,  8.0521e+00,  ..., -5.0653e-01,
          -6.8631e-01, -1.0216e+00]]], grad_fn=<ViewBackward0>)
My attention
tensor([[[ 1.1816e+01, -1.1000e+00,  4.8975e+00,  ..., -1.9040e-01,
           1.4437e-01, -7.6243e-01],
         [ 2.1397e+01,  3.3469e+00, -1.4271e+00,  ...,  1.2569e+00,
          -1.4903e+00, -2.0287e-03],
         [-8.0171e+00, -3.6174e+00,  3.2031e+00,  ...,  8.7513e-01,
          -9.2649e-01,  5.2257e-01],
         [-7.1662e+00, -2.0880e+01,  1.4626e+01,  ...,  9.2104e-01,
           9.043

#### Comparing MLP

In [13]:
print("MLP")
print("GPT2 MLP")
print(gpt2_block.mlp(x))
print("My MLP")
print(my_block.mlp_block(x))

print(t.allclose(my_block.mlp_block(x), gpt2_block.mlp(x), atol=2e-2))

MLP
GPT2 MLP
tensor([[[ 2.0432e+01, -7.8817e-01, -8.1349e+00,  ...,  8.2477e+00,
           8.5021e+00, -6.4954e+00],
         [ 1.1286e+01, -6.0089e+00, -1.3163e+01,  ..., -2.5606e-01,
           1.3157e+01,  7.2588e+00],
         [-2.0909e-02,  2.0334e-02,  3.1577e+00,  ...,  5.4568e+00,
          -1.6100e+01, -1.0993e+01],
         [ 1.1805e+01, -1.0026e+00, -1.9776e+01,  ...,  2.2704e+01,
           5.3657e+00, -9.5716e-03],
         [ 5.8687e+00, -1.5317e+00, -1.6565e+01,  ..., -4.3700e+00,
           3.3641e+00,  7.3793e-01]]], grad_fn=<ViewBackward0>)
My MLP
tensor([[[ 2.0431e+01, -7.8880e-01, -8.1335e+00,  ...,  8.2477e+00,
           8.5018e+00, -6.4947e+00],
         [ 1.1286e+01, -6.0088e+00, -1.3162e+01,  ..., -2.5659e-01,
           1.3156e+01,  7.2580e+00],
         [-2.1170e-02,  1.9706e-02,  3.1581e+00,  ...,  5.4568e+00,
          -1.6099e+01, -1.0994e+01],
         [ 1.1806e+01, -1.0025e+00, -1.9776e+01,  ...,  2.2704e+01,
           5.3651e+00, -9.9052e-03],
        

#### Comparing GeLU

In [14]:
from src.activations import gelu
from torch.nn.functional import gelu as torch_gelu

print("GeLU")
x = t.randn((10,))

print("GPT2 GeLU")
print(gpt2_block.mlp.act(x))
print("My GeLU")
print(gelu(x))
print("Torch GeLU")
print(torch_gelu(x))


GeLU
GPT2 GeLU
tensor([-0.1293,  0.7659, -0.1003, -0.0208,  0.1889, -0.1695, -0.0808, -0.1481,
         1.5389, -0.1255])
My GeLU
tensor([-0.1293,  0.7660, -0.1001, -0.0212,  0.1889, -0.1694, -0.0808, -0.1479,
         1.5391, -0.1253])
Torch GeLU
tensor([-0.1293,  0.7660, -0.1001, -0.0212,  0.1889, -0.1694, -0.0808, -0.1479,
         1.5391, -0.1253])


#### Comparing MLP using GPT2 GeLU

In [18]:
print("My MLP with their GELU")
x = t.randn((1, 10, 768))

print("GPT2 MLP Block")
gpt2_mlp = gpt2_block.mlp(x)
print(gpt2_mlp)

print("My MLP Block with GPT2 GeLU")
my_mlp = my_block.mlp_block.linear2(gpt2_block.mlp.act(my_block.mlp_block.linear1(x)))
print(my_mlp)

print(t.allclose(gpt2_mlp, my_mlp, atol=1e-4))

print("The main difference between our models is in the MLP block, because OpenAI used a slightly different version of GeLU.")

My MLP with their GELU
GPT2 MLP Block
tensor([[[ 16.8194,   7.7999, -21.8079,  ...,   2.2190,   7.4548,  -5.9318],
         [ -0.8979,   0.6438, -18.4866,  ...,  16.4639,   9.0948,  14.2512],
         [ 11.9351, -38.4903,  -8.8888,  ...,  -7.8183,  14.3466,  28.1684],
         ...,
         [ -8.6321,  -7.5341,  -9.3735,  ...,  18.3597,  -2.0436,   2.5461],
         [ 13.7143,   8.5250, -17.8349,  ...,   9.8425,  -0.4575,  11.3794],
         [ 13.0540,   5.1588,  -5.2415,  ...,  -1.3216,  -3.6718,  20.2367]]],
       grad_fn=<ViewBackward0>)
My MLP Block with GPT2 GeLU
tensor([[[ 16.8194,   7.7999, -21.8079,  ...,   2.2190,   7.4548,  -5.9318],
         [ -0.8979,   0.6438, -18.4866,  ...,  16.4639,   9.0948,  14.2512],
         [ 11.9351, -38.4903,  -8.8888,  ...,  -7.8183,  14.3466,  28.1684],
         ...,
         [ -8.6321,  -7.5341,  -9.3735,  ...,  18.3597,  -2.0436,   2.5461],
         [ 13.7143,   8.5250, -17.8349,  ...,   9.8425,  -0.4575,  11.3794],
         [ 13.0540,   5.1

## Comparing GPT2 embedding with my implementation of the GPT2 embedding

In [29]:
from src.models import GPT2SmallModel
my_model = GPT2SmallModel()

from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')

my_model.embedding_layer.position_embedding_layer.weight = model.wpe.weight
my_model.embedding_layer.token_embedding_layer.weight = model.wte.weight

text = "This is a test sentence."
tokens = t.tensor(tokenizer(text)['input_ids']).unsqueeze(0)
tokens

tensor([[1212,  318,  257, 1332, 6827,   13]])

In [30]:
print("Embedding outputs")
print("GPT2 embedding output")
gpt2_emb_out = model.wte(tokens) + model.wpe(t.arange(tokens.shape[-1]))
print(gpt2_emb_out)
print("My embedding output")
my_emb_out = my_model.embedding_layer(tokens)
print(my_emb_out)

t.allclose(gpt2_emb_out, my_emb_out)

Embedding outputs
GPT2 embedding output
tensor([[[ 0.0065, -0.2930,  0.0762,  ...,  0.0184, -0.0275,  0.1638],
         [ 0.0142, -0.0437, -0.0393,  ...,  0.1487, -0.0278, -0.0255],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         [-0.0580,  0.0095,  0.2207,  ..., -0.0635,  0.0760, -0.0543],
         [-0.0888, -0.0326,  0.1666,  ..., -0.2539, -0.0370, -0.2046],
         [ 0.0562, -0.0452,  0.1596,  ..., -0.0676,  0.0567,  0.0888]]],
       grad_fn=<AddBackward0>)
My embedding output
tensor([[[ 0.0065, -0.2930,  0.0762,  ...,  0.0184, -0.0275,  0.1638],
         [ 0.0142, -0.0437, -0.0393,  ...,  0.1487, -0.0278, -0.0255],
         [-0.0464, -0.0791,  0.1016,  ...,  0.0623,  0.0928, -0.0598],
         [-0.0580,  0.0095,  0.2207,  ..., -0.0635,  0.0760, -0.0543],
         [-0.0888, -0.0326,  0.1666,  ..., -0.2539, -0.0370, -0.2046],
         [ 0.0562, -0.0452,  0.1596,  ..., -0.0676,  0.0567,  0.0888]]],
       grad_fn=<AddBackward0>)


True

## Comparing GPT2 model vs. my implementation of the GPT2 model

In [31]:
from src.models import GPT2SmallModel
my_model = GPT2SmallModel()
my_model.eval()

# Importing GPT2LMHeadModel instead of GPT2Model because it applies the final unembed to get logits
from transformers import AutoTokenizer, GPT2LMHeadModel
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("openai-community/gpt2")
gpt2_model.eval()

text = "This is a test sentence."
tokens = t.tensor(tokenizer(text)['input_ids']).unsqueeze(0)
tokens

tensor([[1212,  318,  257, 1332, 6827,   13]])

In [32]:
# Copying over weights
my_model.embedding_layer.position_embedding_layer.weight = gpt2_model.transformer.wpe.weight
my_model.embedding_layer.token_embedding_layer.weight = gpt2_model.transformer.wte.weight

my_model.final_ln.weight, my_model.final_ln.bias = gpt2_model.transformer.ln_f.weight, gpt2_model.transformer.ln_f.bias

my_model_block_list = my_model.transformer_blocks
gpt2_model_block_list = gpt2_model.transformer.h

for my_block, gpt2_block in zip(my_model_block_list, gpt2_model_block_list):
    my_block.norm_layer1.weight, my_block.norm_layer1.bias = gpt2_block.ln_1.weight, gpt2_block.ln_1.bias
    my_block.norm_layer2.weight, my_block.norm_layer2.bias = gpt2_block.ln_2.weight, gpt2_block.ln_2.bias

    my_block.mlp_block.linear1.weight, my_block.mlp_block.linear1.bias = t.nn.Parameter(gpt2_block.mlp.c_fc.weight.T), gpt2_block.mlp.c_fc.bias
    my_block.mlp_block.linear2.weight, my_block.mlp_block.linear2.bias = t.nn.Parameter(gpt2_block.mlp.c_proj.weight.T), gpt2_block.mlp.c_proj.bias

    (wq,wk,wv) = t.chunk(gpt2_block.attn.c_attn.weight.T, 3, dim=0)
    (bq,bk,bv) = t.chunk(gpt2_block.attn.c_attn.bias, 3, dim=0)

    my_block.mha_block.linear_q.weight, my_block.mha_block.linear_q.bias = t.nn.Parameter(wq), t.nn.Parameter(bq)
    my_block.mha_block.linear_k.weight, my_block.mha_block.linear_k.bias = t.nn.Parameter(wk), t.nn.Parameter(bk)
    my_block.mha_block.linear_v.weight, my_block.mha_block.linear_v.bias = t.nn.Parameter(wv), t.nn.Parameter(bv)
    my_block.mha_block.linear_o.weight, my_block.mha_block.linear_o.bias = t.nn.Parameter(gpt2_block.attn.c_proj.weight.T), gpt2_block.attn.c_proj.bias


### Logit outputs

In [33]:
print("Model logits")
print("GPT2 model logits")
gpt2_logits = gpt2_model(**tokenizer(text, return_tensors='pt')).logits
print(gpt2_logits)
print("My model logits")
my_logits = my_model(tokens)
print(my_logits)

print(t.allclose(gpt2_logits, my_logits, atol=1e-1))

Model logits
GPT2 model logits
tensor([[[ -35.8890,  -35.2049,  -39.1336,  ...,  -42.4869,  -41.8197,
           -36.0383],
         [-107.7291, -108.0175, -113.2967,  ..., -116.4645, -115.7443,
          -110.8654],
         [-111.7507, -111.5704, -114.5443,  ..., -120.7242, -117.1756,
          -112.3996],
         [ -86.1846,  -88.5057,  -94.3530,  ..., -101.3573,  -98.6974,
           -91.1616],
         [-106.4531, -108.7300, -115.4155,  ..., -119.6631, -119.1774,
          -110.7877],
         [-146.7139, -145.9828, -146.9487,  ..., -155.2113, -158.0557,
          -139.4035]]], grad_fn=<UnsafeViewBackward0>)
My model logits
tensor([[[ -35.8260,  -35.1460,  -39.0735,  ...,  -42.4222,  -41.7547,
           -35.9785],
         [-107.7155, -108.0055, -113.2770,  ..., -116.4519, -115.7324,
          -110.8482],
         [-111.7447, -111.5662, -114.5354,  ..., -120.7213, -117.1712,
          -112.3930],
         [ -86.1794,  -88.5035,  -94.3420,  ..., -101.3490,  -98.6924,
           -

### Argmax sampling

In [34]:
text = "The Large Apple, by Roald Dahl. \nLate at night,"
tokens_gpt2 = tokenizer(text, return_tensors='pt')
tokens_curr = tokenizer(text, return_tensors='pt')['input_ids']

tokens_gpt2

{'input_ids': tensor([[  464, 13601,  4196,    11,   416,  5564,  1940, 41471,    13,   220,
           198, 26302,   379,  1755,    11]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [35]:
for i in range(10):
    gpt2_next_token = gpt2_model(**tokens_gpt2).logits[0][-1].argmax()
    my_model_next_token = my_model(tokens_curr)[0][-1].argmax()

    tokens_gpt2['input_ids'] = t.cat([tokens_gpt2['input_ids'], gpt2_next_token.reshape((1,1))], dim=1)
    tokens_gpt2['attention_mask'] = t.cat([tokens_gpt2['attention_mask'], t.tensor(1).reshape((1,1))], dim=1)
    tokens_curr = t.cat([tokens_curr, my_model_next_token.reshape((1,1))], dim=1)

In [36]:
print("GPT2 argmax output")
print(''.join(tokenizer.convert_ids_to_tokens(tokens_gpt2['input_ids'][0])).replace('Ġ', ' ').replace('Ċ', '\n'))
print()
print("My implementation output")
print(''.join(tokenizer.convert_ids_to_tokens(tokens_curr[0])).replace('Ġ', ' ').replace('Ċ', '\n'))

GPT2 argmax output
The Large Apple, by Roald Dahl. 
Late at night, I was sitting in the living room of my home

My implementation output
The Large Apple, by Roald Dahl. 
Late at night, I was sitting in the living room of my home


## Normal sampling from my version of the GPT-2 model

In [40]:
# https://openai.com/index/better-language-models/
text = "In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English."
tokens = tokenizer(text, return_tensors='pt')['input_ids']

tokens

tensor([[  818,   257, 14702,  4917,    11, 11444,  5071,   257, 27638,   286,
         28000, 19942,  2877,   287,   257,  6569,    11,  4271, 31286,  1850,
         19272,    11,   287,   262,   843,   274, 21124,    13,  3412,   517,
          6452,   284,   262,  4837,   373,   262,  1109,   326,   262, 28000,
         19942,  5158,  2818,  3594,    13]])

In [41]:
for i in range(100):
    my_model_next_token = t.multinomial(t.softmax(my_model(tokens)[0][-1], dim=0), 1)

    tokens = t.cat([tokens, my_model_next_token.unsqueeze(0)], dim=1)

In [39]:
print("GPT2 completion of unicorn prompt")
print(''.join(tokenizer.convert_ids_to_tokens(tokens[0])).replace('Ġ', ' ').replace('Ċ', '\n'))

GPT2 completion of unicorn prompt
In a shocking finding, scientist discovered a herd of unicorns living in a remote, previously unexplored valley, in the Andes Mountains. Even more surprising to the researchers was the fact that the unicorns spoke perfect English. This clearly opens the door to potential insights into human language.

Warwicz had his humble beginnings in the fall of 1948 as a visiting missionary at Judeo-Christian monastery in Guatemala City, growing up in the tightly knit neighborhoods of Bolivian refugee camps, living close to his ancestral families, where his desire to learn had never been burned. Lacking a university education, he passed Jeremiah B. McCaw in segregation and Catholic school records, and was the youngest of three children. He
